# 📊 Forex Data Download & Feature Engineering
## Machine Learning for Forex Trading Research

**Notebook này bao gồm:**
1. Tải dữ liệu Forex từ yfinance (2020-2025)
2. Feature Engineering với 110+ features
3. Lưu dataset sẵn sàng cho ML

---

## 1. Cài Đặt Thư Viện

In [ ]:
!pip install yfinance pandas numpy -q
print("✅ Đã cài đặt thư viện!")

## 2. Import Thư Viện

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import warnings
import time
warnings.filterwarnings('ignore')

print("✅ Import thành công!")

## 3. Cấu Hình

In [ ]:
# ============================================
# CẤU HÌNH
# ============================================

# 5 cặp tiền Major đại diện
CURRENCY_PAIRS = {
    'EURUSD=X': 'EUR_USD',   # Euro vs US Dollar
    'GBPUSD=X': 'GBP_USD',   # British Pound vs US Dollar
    'USDJPY=X': 'USD_JPY',   # US Dollar vs Japanese Yen
    'AUDUSD=X': 'AUD_USD',   # Australian Dollar vs US Dollar
    'USDCHF=X': 'USD_CHF'    # US Dollar vs Swiss Franc
}

# Thời gian nghiên cứu
START_DATE = '2020-01-01'
END_DATE = '2025-12-31'

# Tạo thư mục
os.makedirs('data/raw/daily', exist_ok=True)
os.makedirs('data/raw/4h', exist_ok=True)
os.makedirs('data/raw/1h', exist_ok=True)
os.makedirs('data/processed/daily', exist_ok=True)
os.makedirs('data/processed/4h', exist_ok=True)
os.makedirs('data/processed/1h', exist_ok=True)

print("✅ Cấu hình hoàn tất!")
print(f"📅 Thời gian: {START_DATE} đến {END_DATE}")
print(f"💱 Số cặp tiền: {len(CURRENCY_PAIRS)}")

---
# PHẦN 1: TẢI DỮ LIỆU
---

## 4. Hàm Tải Dữ Liệu Daily

In [ ]:
def download_daily_data(ticker, name):
    """Tải dữ liệu khung Daily từ 2020"""
    print(f"  📥 Đang tải {name} Daily...")
    
    try:
        data = yf.download(
            ticker, 
            start=START_DATE, 
            end=END_DATE, 
            interval='1d',
            progress=False
        )
        
        if len(data) == 0:
            print(f"  ⚠️  Không có dữ liệu cho {name}")
            return None
        
        # Làm sạch dữ liệu
        data = data.reset_index()
        
        # Đổi tên cột nếu là MultiIndex
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]
        
        # Chuẩn hóa tên cột
        data.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
        data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        # Lưu file
        output_path = f'data/raw/daily/{name}_daily.csv'
        data.to_csv(output_path, index=False)
        print(f"  ✅ Đã lưu {name} Daily: {len(data)} records")
        
        return data
        
    except Exception as e:
        print(f"  ❌ Lỗi khi tải {name}: {str(e)}")
        return None

## 5. Hàm Tải Dữ Liệu 1H (Chunked)

In [ ]:
def download_hourly_data(ticker, name):
    """
    Tải dữ liệu 1H
    Lưu ý: yfinance giới hạn dữ liệu 1h trong 730 ngày gần nhất
    """
    print(f"  📥 Đang tải {name} 1H...")
    
    try:
        # yfinance chỉ cho phép tải 730 ngày với interval 1h
        end_date = datetime.now()
        start_date = end_date - timedelta(days=729)
        
        data = yf.download(
            ticker,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            interval='1h',
            progress=False
        )
        
        if len(data) == 0:
            print(f"  ⚠️  Không có dữ liệu 1H cho {name}")
            return None
        
        # Làm sạch dữ liệu
        data = data.reset_index()
        
        # Đổi tên cột nếu là MultiIndex
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]
        
        # Chuẩn hóa tên cột
        if 'Datetime' in data.columns:
            data.rename(columns={'Datetime': 'Date'}, inplace=True)
        
        data.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
        data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        # Lưu file 1H
        output_path = f'data/raw/1h/{name}_1h.csv'
        data.to_csv(output_path, index=False)
        print(f"  ✅ Đã lưu {name} 1H: {len(data)} records")
        
        return data
        
    except Exception as e:
        print(f"  ❌ Lỗi khi tải {name} 1H: {str(e)}")
        return None

## 6. Hàm Resample sang 4H

In [ ]:
def resample_to_4h(data_1h, name):
    """Chuyển đổi dữ liệu 1H thành 4H"""
    if data_1h is None or len(data_1h) == 0:
        return None
    
    print(f"  🔄 Đang chuyển đổi {name} sang 4H...")
    
    try:
        df = data_1h.copy()
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        
        # Resample sang 4H
        data_4h = df.resample('4h').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }).dropna()
        
        data_4h = data_4h.reset_index()
        
        # Lưu file 4H
        output_path = f'data/raw/4h/{name}_4h.csv'
        data_4h.to_csv(output_path, index=False)
        print(f"  ✅ Đã lưu {name} 4H: {len(data_4h)} records")
        
        return data_4h
        
    except Exception as e:
        print(f"  ❌ Lỗi khi chuyển đổi {name} sang 4H: {str(e)}")
        return None

## 7. 🚀 Tải Toàn Bộ Dữ Liệu

In [ ]:
print("=" * 60)
print("🚀 BẮT ĐẦU TẢI DỮ LIỆU FOREX")
print(f"📅 Thời gian: {START_DATE} đến {END_DATE}")
print(f"💱 Số cặp tiền: {len(CURRENCY_PAIRS)}")
print("=" * 60)

# Lưu trữ dữ liệu
all_data = {}

# Tải dữ liệu cho từng cặp tiền
for ticker, name in CURRENCY_PAIRS.items():
    print(f"\n📊 Đang xử lý {name}...")
    
    all_data[name] = {}
    
    # Tải Daily
    all_data[name]['daily'] = download_daily_data(ticker, name)
    
    # Tải 1H
    all_data[name]['1h'] = download_hourly_data(ticker, name)
    
    # Tạo 4H từ 1H
    if all_data[name]['1h'] is not None:
        all_data[name]['4h'] = resample_to_4h(all_data[name]['1h'], name)

print("\n" + "=" * 60)
print("✅ HOÀN THÀNH TẢI DỮ LIỆU!")
print("=" * 60)

---
# PHẦN 2: FEATURE ENGINEERING
---

## 8. Tham Số Feature Engineering

In [ ]:
# Tham số cho indicators
RSI_PERIOD = 14
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
BB_PERIOD = 20
BB_STD = 2
ATR_PERIOD = 14
STOCH_PERIOD = 14
LAG_PERIODS = [1, 2, 3, 5, 7, 14, 21]
MA_PERIODS = [7, 14, 21, 50, 100, 200]

print("✅ Tham số đã cấu hình!")

## 9. Hàm Tính Technical Indicators

In [ ]:
def calculate_rsi(close, period=14):
    """Tính RSI"""
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(close, fast=12, slow=26, signal=9):
    """Tính MACD"""
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

def calculate_bollinger_bands(close, period=20, std_dev=2):
    """Tính Bollinger Bands"""
    sma = close.rolling(window=period).mean()
    std = close.rolling(window=period).std()
    upper = sma + (std * std_dev)
    lower = sma - (std * std_dev)
    width = (upper - lower) / sma
    position = (close - lower) / (upper - lower)
    return upper, sma, lower, width, position

def calculate_atr(high, low, close, period=14):
    """Tính ATR"""
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def calculate_stochastic(high, low, close, period=14):
    """Tính Stochastic"""
    lowest = low.rolling(window=period).min()
    highest = high.rolling(window=period).max()
    k = 100 * (close - lowest) / (highest - lowest)
    d = k.rolling(window=3).mean()
    return k, d

print("✅ Các hàm Technical Indicators đã sẵn sàng!")

## 10. Hàm Feature Engineering Chính

In [ ]:
def add_all_features(df):
    """
    Thêm tất cả features vào DataFrame
    Input: DataFrame với cột Date, Open, High, Low, Close, Volume
    Output: DataFrame với ~110+ features
    """
    df = df.copy()
    
    # ========================
    # PRICE FEATURES
    # ========================
    df['Return'] = df['Close'].pct_change()
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Price_Change'] = df['Close'] - df['Open']
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    df['Range'] = df['High'] - df['Low']
    df['Body'] = abs(df['Close'] - df['Open'])
    df['Upper_Shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['Lower_Shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    df['Candle_Color'] = np.where(df['Close'] > df['Open'], 1, 0)
    df['Gap'] = df['Open'] - df['Close'].shift(1)
    
    # Volatility
    for p in [5, 10, 20]:
        df[f'Volatility_{p}'] = df['Return'].rolling(window=p).std()
    
    # ========================
    # TECHNICAL INDICATORS
    # ========================
    # RSI
    df['RSI'] = calculate_rsi(df['Close'], RSI_PERIOD)
    df['RSI_overbought'] = (df['RSI'] > 70).astype(int)
    df['RSI_oversold'] = (df['RSI'] < 30).astype(int)
    
    # MACD
    df['MACD'], df['MACD_signal'], df['MACD_histogram'] = calculate_macd(
        df['Close'], MACD_FAST, MACD_SLOW, MACD_SIGNAL
    )
    df['MACD_crossover'] = np.where(df['MACD'] > df['MACD_signal'], 1, -1)
    
    # Bollinger Bands
    df['BB_upper'], df['BB_middle'], df['BB_lower'], df['BB_width'], df['BB_position'] = \
        calculate_bollinger_bands(df['Close'], BB_PERIOD, BB_STD)
    
    # ATR
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'], ATR_PERIOD)
    df['ATR_percent'] = df['ATR'] / df['Close'] * 100
    
    # Stochastic
    df['Stoch_K'], df['Stoch_D'] = calculate_stochastic(
        df['High'], df['Low'], df['Close'], STOCH_PERIOD
    )
    
    # Momentum & ROC
    df['Momentum_10'] = df['Close'] - df['Close'].shift(10)
    df['ROC_10'] = ((df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10)) * 100
    
    # Moving Averages
    for p in MA_PERIODS:
        if len(df) > p:
            df[f'SMA_{p}'] = df['Close'].rolling(window=p).mean()
            df[f'EMA_{p}'] = df['Close'].ewm(span=p, adjust=False).mean()
            df[f'Close_vs_SMA_{p}'] = (df['Close'] / df[f'SMA_{p}'] - 1) * 100
    
    # MA Crossovers
    if 'SMA_7' in df.columns and 'SMA_21' in df.columns:
        df['MA_7_21_cross'] = np.where(df['SMA_7'] > df['SMA_21'], 1, -1)
    if 'SMA_50' in df.columns and 'SMA_200' in df.columns:
        df['MA_50_200_cross'] = np.where(df['SMA_50'] > df['SMA_200'], 1, -1)
    
    # ========================
    # LAGGED FEATURES
    # ========================
    for lag in LAG_PERIODS:
        if len(df) > lag:
            df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
            df[f'Return_lag_{lag}'] = df['Return'].shift(lag)
    
    # RSI lags
    for lag in [1, 3, 5]:
        df[f'RSI_lag_{lag}'] = df['RSI'].shift(lag)
    
    # Cumulative Returns
    for p in [3, 5, 10, 20]:
        if len(df) > p:
            df[f'Cumulative_Return_{p}'] = df['Close'].pct_change(periods=p)
    
    # Rolling Stats
    for p in [5, 10, 20]:
        if len(df) > p:
            df[f'Rolling_Mean_{p}'] = df['Close'].rolling(window=p).mean()
            df[f'Rolling_Std_{p}'] = df['Close'].rolling(window=p).std()
            df[f'Rolling_Min_{p}'] = df['Close'].rolling(window=p).min()
            df[f'Rolling_Max_{p}'] = df['Close'].rolling(window=p).max()
    
    # ========================
    # TIME FEATURES
    # ========================
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day_of_Week'] = df['Date'].dt.dayofweek
    df['Day_of_Month'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Quarter'] = df['Date'].dt.quarter
    df['Year'] = df['Date'].dt.year
    df['Week_of_Year'] = df['Date'].dt.isocalendar().week.astype(int)
    
    # Binary time features
    df['Is_Monday'] = (df['Day_of_Week'] == 0).astype(int)
    df['Is_Friday'] = (df['Day_of_Week'] == 4).astype(int)
    df['Is_Month_Start'] = df['Date'].dt.is_month_start.astype(int)
    df['Is_Month_End'] = df['Date'].dt.is_month_end.astype(int)
    
    # Cyclical encoding
    df['Day_sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
    df['Day_cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    
    # ========================
    # TARGET VARIABLES
    # ========================
    # Classification: Direction
    df['Target_Direction'] = np.where(
        df['Close'].shift(-1) > df['Close'], 1, 0
    )
    
    # Regression: Next Return
    df['Target_Return'] = df['Close'].pct_change().shift(-1)
    
    # Regression: Next Close
    df['Target_Close'] = df['Close'].shift(-1)
    
    return df

print("✅ Hàm Feature Engineering đã sẵn sàng!")

## 11. 🚀 Thực Hiện Feature Engineering

In [ ]:
print("=" * 60)
print("🚀 BẮT ĐẦU FEATURE ENGINEERING")
print("=" * 60)

timeframes = ['daily', '4h', '1h']

processed_data = {}

for name in CURRENCY_PAIRS.values():
    processed_data[name] = {}
    
    for tf in timeframes:
        input_path = f'data/raw/{tf}/{name}_{tf}.csv'
        output_path = f'data/processed/{tf}/{name}_{tf}_processed.csv'
        
        if os.path.exists(input_path):
            print(f"\n📊 Đang xử lý {name} ({tf})...")
            
            # Đọc dữ liệu
            df = pd.read_csv(input_path)
            original_cols = len(df.columns)
            original_rows = len(df)
            
            # Feature Engineering
            df_processed = add_all_features(df)
            
            # Loại bỏ NaN
            df_clean = df_processed.dropna()
            
            # Lưu file
            df_clean.to_csv(output_path, index=False)
            
            processed_data[name][tf] = df_clean
            
            print(f"  ✅ {original_cols} → {len(df_clean.columns)} columns")
            print(f"  ✅ {original_rows} → {len(df_clean)} rows")
            print(f"  💾 Đã lưu: {output_path}")

print("\n" + "=" * 60)
print("✅ FEATURE ENGINEERING HOÀN THÀNH!")
print("=" * 60)

---
# PHẦN 3: KIỂM TRA KẾT QUẢ
---

## 12. Xem Mẫu Dữ Liệu

In [ ]:
# Đọc một file mẫu
sample_df = pd.read_csv('data/processed/daily/EUR_USD_daily_processed.csv')

print(f"📊 Kích thước dữ liệu: {sample_df.shape}")
print(f"📊 Số features: {len(sample_df.columns)}")
print("\n📋 Danh sách các cột:")
print(sample_df.columns.tolist())

In [ ]:
# Xem 5 dòng đầu
sample_df.head()

In [ ]:
# Thống kê mô tả
sample_df.describe()

## 13. Phân Phối Target Variable

In [ ]:
# Distribution của Target_Direction
print("Phân phối Target_Direction:")
print(sample_df['Target_Direction'].value_counts())
print(f"\nTỷ lệ Up/Down: {sample_df['Target_Direction'].mean()*100:.2f}% Up")

## 14. Liệt Kê Tất Cả Files

In [ ]:
import os

print("📁 CẤU TRÚC THƯ MỤC DỮ LIỆU:")
print("=" * 50)

for root, dirs, files in os.walk('data'):
    level = root.replace('data', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}📂 {os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        if file.endswith('.csv'):
            filepath = os.path.join(root, file)
            size_mb = os.path.getsize(filepath) / (1024 * 1024)
            print(f'{subindent}📄 {file} ({size_mb:.2f} MB)')

---
# PHẦN 4: CHUẨN BỊ CHO ML
---

## 15. Hàm Load Data cho ML

In [ ]:
def load_data_for_ml(pair='EUR_USD', timeframe='daily'):
    """
    Load dữ liệu đã xử lý cho ML
    
    Parameters:
    - pair: 'EUR_USD', 'GBP_USD', 'USD_JPY', 'AUD_USD', 'USD_CHF'
    - timeframe: 'daily', '4h', '1h'
    
    Returns:
    - X: Features
    - y_class: Target cho Classification
    - y_reg: Target cho Regression
    """
    filepath = f'data/processed/{timeframe}/{pair}_{timeframe}_processed.csv'
    df = pd.read_csv(filepath)
    
    # Loại bỏ các cột không phải features
    exclude_cols = ['Date', 'Target_Direction', 'Target_Return', 'Target_Close']
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    X = df[feature_cols]
    y_class = df['Target_Direction']
    y_reg = df['Target_Return']
    
    print(f"✅ Loaded {pair} ({timeframe})")
    print(f"   Features: {X.shape[1]}")
    print(f"   Samples: {X.shape[0]}")
    
    return X, y_class, y_reg

print("✅ Hàm load_data_for_ml đã sẵn sàng!")

In [ ]:
# Ví dụ load data
X, y_class, y_reg = load_data_for_ml('EUR_USD', 'daily')

## 16. Download Files về Máy (Tùy chọn)

In [ ]:
# Nén và download tất cả data
!zip -r forex_data.zip data/

from google.colab import files
files.download('forex_data.zip')

print("✅ Đã tạo file forex_data.zip để download!")

---
# ✅ HOÀN THÀNH!

Dataset đã sẵn sàng để train các models ML:
- **Classification**: Random Forest, XGBoost, LSTM
- **Regression**: Linear Regression, Gradient Boosting

---